In [1]:
import sklearn
import cv2
import numpy as np

Aanvankelijk waren dit de in- en outputs die ik in gedachten had voor het model:

**In:**
* zoom, resolutie, mean kleur foto, kleurverdeling (histogram)

**Out:**
* cell threshold, min_area, adaptive threshold, threshold/blur stippen


Om technische redenen (resolutie bestaat uit 2 cijfers) en om het model zo klein mogelijk te houden, zodat het op minder data getrained kan worden heb ik het oorspronkelijke plan naar dit veranderd:

**In:**
* schaal van foto, hoogte foto, breedte foto, mean kleur foto, contrast

**Out:**
* min_area, dot_thresh

### Dit is de data:

In [8]:
paths = ['data/Complemented\\2024-08i compl OADChi E2_17.tif','data/Mutant/2024-08i mutant Mtb 2nd exp_D4_02.tif','data/Mutant/2024-08i_mutant Mtb_D5_18.tif','data/Complemented/2024-08i compl OADChi E2_02.tif','data/Mutant/2024-08i_mutant Mtb_D5_17.tif',
         'data\Mutant/2024-08i mutant Mtb 2nd exp_D4_01.tif','data/Wildtype/2024_08i_WT_D3_OADChi_05.tif','data/Wildtype/2024_08i_WT_D3_OADChi_07.tif','data/Wildtype/2024_08i_WT_D3_OADChi_20.tif','data/Wildtype/2024-08i WT Mtb 2nd exp_D2_01.tif',
         'data/Wildtype/2024-08i WT Mtb 2nd exp_D2_05.tif','data/Wildtype/2024-08i WT Mtb 2nd exp_D2_12.tif','data/Wildtype/2024-08i WT Mtb 2nd exp_D2_14.tif','data/Wildtype/2024-08i WT Mtb 2nd exp_D2_20.tif','data/Wildtype/2024-08i WT Mtb 2nd exp_D2_23.tif']

In [ ]:
y = [[30,40],[120,35],[40,45],[60,45],[80,45],[80,45],[100,45],[100,45],[90,45],[90,45],[50,45],[40,45],[100,45],[40,45],[40,45]]

Het viel me tijdens het maken van de data dat de dot_thresh vrijwel geen invloed heeft, en alle foto's op `45` werken. dot_tresh ga ik daarom ook verwijderen uit de data.

In [15]:
y_fixed = []
for i in y:
    j = i[0]
    y_fixed.append(j)
y = y_fixed

De output van het model is nu alleen nog maar min_area

De volgende stap is de input van het model maken. Er is een lijst met alle bestanden, `paths`. Bij elk bestand in deze lijst verzameld de volgende functie `img_info()` wat informatie.

In [16]:
from bar_and_scale_finder import scale_finder

def img_info(image_path):
    image = cv2.imread(image_path)
    mean = float(image.mean())
    h,w =  image.shape[0:2]
    contrast = float(np.std(image))
    return [scale_finder(image_path), h,w, mean, contrast]
x=[]
for path in paths:
    x.append(img_info(path))

Neither CUDA nor MPS are available - defaulting to CPU. Note: This module is much faster with a GPU.
d:\HW\gold_nanoparticle_TC\.venv\Lib\site-packages\torch\utils\data\dataloader.py:665: UserWarning: 'pin_memory' argument is set as true but no accelerator is found, then device pinned memory won't be used.
  warnings.warn(warn_msg)
Neither CUDA nor MPS are available - defaulting to CPU. Note: This module is much faster with a GPU.
Neither CUDA nor MPS are available - defaulting to CPU. Note: This module is much faster with a GPU.
Neither CUDA nor MPS are available - defaulting to CPU. Note: This module is much faster with a GPU.
Neither CUDA nor MPS are available - defaulting to CPU. Note: This module is much faster with a GPU.
Neither CUDA nor MPS are available - defaulting to CPU. Note: This module is much faster with a GPU.
Neither CUDA nor MPS are available - defaulting to CPU. Note: This module is much faster with a GPU.
Neither CUDA nor MPS are available - defaulting to CPU. Note

In [17]:
print(x)

[[0.6968641114982579, 3840, 5120, 183.77602771335177, 32.8951779909937], [0.5025125628140703, 3840, 5120, 178.83489262898763, 40.37792014225524], [0.5025125628140703, 3840, 5120, 192.97049353705512, 57.574980464458285], [0.6968641114982579, 3840, 5120, 208.19448016696506, 36.15486249758521], [0.5025125628140703, 3840, 5120, 206.93722176445854, 40.819356963800224], [0.5025125628140703, 3840, 5120, 182.78424994574652, 35.88123807627787], [0.5025125628140703, 3840, 5120, 164.8578357272678, 42.03827300355256], [0.5025125628140703, 3840, 5120, 169.83906741672092, 43.291019095331094], [0.5025125628140703, 3840, 5120, 149.3517938232422, 43.80813973241743], [0.5025125628140703, 3840, 5120, 196.80446445041233, 46.801500753918845], [0.6968641114982579, 3840, 5120, 201.0984417046441, 43.30529383623584], [0.6968641114982579, 3840, 5120, 210.16379928588867, 44.52378333994371], [0.5025125628140703, 3840, 5120, 194.17714799669054, 40.695184535128575], [0.6968641114982579, 3840, 5120, 188.135110558403

# Trainen en testen

In [18]:
from sklearn.ensemble import RandomForestClassifier

clf = RandomForestClassifier(n_estimators=10)
clf = clf.fit(x, y)

In [20]:
test = [img_info('data/Mutant/2024-08i_mutant Mtb_D5_37.tif')] #Moet 2d om wat voor reden
print(test)
clf.predict(test)

Neither CUDA nor MPS are available - defaulting to CPU. Note: This module is much faster with a GPU.


[[0.5025125628140703, 3840, 5120, 197.76578053792318, 43.43403970469155]]


d:\HW\gold_nanoparticle_TC\.venv\Lib\site-packages\torch\utils\data\dataloader.py:665: UserWarning: 'pin_memory' argument is set as true but no accelerator is found, then device pinned memory won't be used.
  warnings.warn(warn_msg)


array([90])

De test-voorspelling klopt, 90 is de correcte min_area